In [24]:
import ast
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from opportunistic_planning import processing, visualization
from scipy.stats import friedmanchisquare, wilcoxon

In [2]:
df = pd.read_csv('all_task_environments_2021-03-17.csv', header=0)

In [3]:
with open ('../../model_evaluation/rnn/results/nn_spatialinfo_prequential_summed_2022-01-19.txt',
          'r') as file:
    nn_results = ast.literal_eval(file.read())

In [4]:
len(nn_results)

190

In [5]:
%time distances_dict = processing.generate_distances_dict(df)

CPU times: user 1.19 s, sys: 109 ms, total: 1.29 s
Wall time: 1.18 s


In [6]:
# n=50: 11min 49
# n=100: 21min 51

%time results_sum = processing.calculate_prediction_error(df, distances_dict, error_function='prequential', n=100, dimensions=[[2, 'xy'], [3, 'xyz']])

/home/zauri/files/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


CPU times: user 21min 51s, sys: 565 ms, total: 21min 51s
Wall time: 21min 51s


In [7]:
results_sum

,"c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; xy","c: 1.1; k: 0.0,0.1,1.1; xyz","c: 1.2; k: 0.0,0.1,1.1; xy","c: 1.2; k: 0.0,0.1,1.1; xyz","c: 1.3; k: 0.0,0.1,1.1; xy","c: 1.3; k: 0.0,0.1,1.1; xyz","c: 1.4; k: 0.0,0.1,1.1; xy","c: 1.4; k: 0.0,0.1,1.1; xyz",...,"c: 1.6; k: 0.8,0.9,1.9; xyz","c: 1.7; k: 0.8,0.9,1.9; xy","c: 1.7; k: 0.8,0.9,1.9; xyz","c: 1.8; k: 0.8,0.9,1.9; xy","c: 1.8; k: 0.8,0.9,1.9; xyz","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,pocgkr,0.723,a1
1,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,cgwpcfks,0.785,a3
2,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,kfsfkspwg,0.806,a5
3,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,pfkswkfsococg,0.862,a11
4,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,wptgkfsoc,0.806,a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,3.0,4.0,4.0,3.5,3.0,4.0,4.0,4.0,3.5,3.0,...,4.0,3.0,3.0,3.0,4.0,3.0,3.0,hhsdbg,0.723,v8
186,3.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,...,2.0,2.5,3.0,3.0,2.0,3.0,3.0,hhsdgb,0.723,v9
187,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,...,2.0,1.0,2.0,1.5,1.0,2.0,1.0,hhsgdb,0.723,v10
188,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,...,1.5,2.0,1.0,2.0,2.0,1.0,1.0,hhsgdb,0.723,v11


In [8]:
lowest_mean, lowest_mean_idx, lowest_median, results_median = processing.get_lowest_error(results_sum)

In [9]:
%matplotlib qt

visualization.plot_comparison_to_baselines(results_median, lowest_mean_idx, lowest_median,
                                          save=False, cpt=None, rnn=nn_results)

In [57]:
#processing.save_results(results_sum, 'results/results_2D3D_n50_2022-01-19.csv')

### Plot best model with baselines

In [23]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_sum['sequence'][:-2].values

res = results_sum[lowest_mean_idx[0]][:-2].values
median = [np.nanmedian(res)] * len(res)

x = [x for x in range (0,len(seqs))]

# plot CPT baseline
#plt.scatter(x, cpt, marker='o', s=20, c='dodgerblue', alpha=0.3, 
#            label=str('CPT baseline median: ') + str(round(np.median(cpt),3)))
#plt.plot(x, [np.median(cpt)] * len(x), '-', c='dodgerblue', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
#plt.fill_between(x, cpt, alpha=0.3, color='dodgerblue')


# plot NN baseline
plt.scatter(x, nn_results, marker='o', s=20, c='navy', alpha=0.8, 
            label=str('NN baseline median: ') + str(round(np.median(nn_results),3)))
plt.plot(x, [np.median(nn_results)] * len(x), '-', c='dodgerblue', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, nn_results, alpha=0.3, color='dodgerblue')


# plot scatter + lines for simulations
plt.scatter(x, res, marker='o', s=26, c='darkviolet', alpha=0.8, 
            label=str('model-generated median: ') + str(round(lowest_median,3)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='darkviolet', alpha=0.95, linewidth=2)
#plt.fill_between(x, res, alpha=0.3, color='darkviolet')


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('accumulated prediction error', fontsize=22)
plt.xlabel('sequence', fontsize=22)
#plt.ylim(0.0, 0.51)
plt.title('best model (all), ' + str(lowest_mean_idx[0]), fontsize=24, pad=20)
plt.margins(0.01)

plt.legend(fontsize=20, framealpha=0.8, loc='upper right', markerscale=2.5)

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [25]:
# Get errors for model
list_xy = []

for col in results_sum:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_sum.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))

Average xy: 4.2730246913580245, stdev: 0.20129004376755086, median: 4.219736842105263


In [37]:
# model vs NN
stat, p = wilcoxon(res, nn_results, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 3780.500, p = 0.00159


In [32]:
# get sequences where OPM performs worse than baseline

worse_than_nn = results_sum[lowest_mean_idx[0]].loc[results_sum[lowest_mean_idx[0]] >= 4.0]
worse_than_nn_sequences = [results_sum.at[x, 'sequence'] for x in worse_than_nn.index]

len(worse_than_nn_sequences)

105

In [36]:
# get sequences where NN performs worse than baseline

worse_than_opm = [i for i in nn_results if i > 4.0]

len(worse_than_opm)

56